In [1]:
import os
import glob
import xml.etree.ElementTree as ET
import statistics
from tqdm import tqdm
import pandas as pd

def parse_xml_binding_energy(xml_file):
    """
    Parse an AutoDock-GPU XML file and compute the average binding energy
    from the <rmsd_table> runs.
    
    Returns:
       receptor_code (str): Receptor code extracted from the XML filename.
       ligand_code (str): Ligand code extracted from the XML filename.
       avg_energy (float): Average binding energy from all <run> entries in the RMSD table.
         Returns (None, None, None) if parsing fails or required data is missing.
    """
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        # Use the XML file's own name to obtain receptor and ligand codes.
        base_name = os.path.splitext(os.path.basename(xml_file))[0]  # e.g., "3ert_lig09685"
        parts = base_name.split("_")
        if len(parts) < 2:
            print(f"Warning: Unexpected filename format in {xml_file}")
            return None, None, None
        receptor_code = parts[0]
        ligand_code = parts[1]
        
        # Find the <result> element and the <rmsd_table> within it.
        result_elem = root.find("result")
        if result_elem is None:
            print(f"Warning: No <result> section in {xml_file}")
            return receptor_code, ligand_code, None
        
        rmsd_table_elem = result_elem.find("rmsd_table")
        if rmsd_table_elem is None:
            print(f"Warning: No <rmsd_table> element in {xml_file}")
            return receptor_code, ligand_code, None
        
        energies = []
        for run in rmsd_table_elem.findall("run"):
            binding_energy = run.get("binding_energy")
            if binding_energy is None:
                raise ValueError(f"Missing binding energy in {xml_file}")
            try:
                energy = float(binding_energy)
                energies.append(energy)
            except ValueError:
                raise ValueError(f"Invalid binding energy value in {xml_file}: {binding_energy}")
            
            if len(energies) > 3:
                break
        
        if not energies:
            return receptor_code, ligand_code, None
        
        avg_energy = statistics.mean(energies)
        return receptor_code, ligand_code, avg_energy
    except ET.ParseError as e:
        print(f"Error parsing {xml_file}: {e}")
        return None, None, None

# Set the directory containing the XML files
input_dir = os.path.join("AutodockGPU", "adgpu_output")  # adjust as needed
xml_pattern = os.path.join(input_dir, "*.xml")
xml_files = glob.glob(xml_pattern)

if not xml_files:
    print(f"No XML files found in {input_dir}")

# Dictionary to accumulate energies per receptor-ligand pair.
# Structure: results[ligand_code][receptor_code] = list of energies
results = {}

for xml_file in tqdm(xml_files):
    receptor, ligand, avg_energy = parse_xml_binding_energy(xml_file)
    if receptor is None or ligand is None or avg_energy is None:
        continue
    if ligand not in results:
        results[ligand] = {}
    if receptor not in results[ligand]:
        results[ligand][receptor] = []
    results[ligand][receptor].append(avg_energy)

# Compute the final mean energy for each receptor-ligand pair.
final_data = []
for ligand_code, receptor_dict in results.items():
    for receptor_code, energy_list in receptor_dict.items():
        if energy_list:
            final_avg = statistics.mean(energy_list)
            final_data.append({
                "Ligand": ligand_code,
                "Receptor": receptor_code,
                "Binding Energy": final_avg
            })
            # print(f"Ligand {ligand_code}, Receptor {receptor_code}: energies = {energy_list} -> Mean = {final_avg:.2f} kcal/mol")

# Create a DataFrame from the final data.
df = pd.DataFrame(final_data)

if not df.empty:
    # Pivot the DataFrame: rows are ligand codes; columns are receptor codes.
    df_pivot = df.pivot(index="Ligand", columns="Receptor", values="Binding Energy")
    df_pivot.sort_index(inplace=True)
    df_pivot = df_pivot[sorted(df_pivot.columns)]
    
    # Now, map receptor codes to receptor names by scanning the receptor folder.
    receptor_dir = "AutodockGPU/receptor"
    receptor_mapping = {}
    
    if os.path.exists(receptor_dir):
        for folder in os.listdir(receptor_dir):
            folder_path = os.path.join(receptor_dir, folder)
            if os.path.isdir(folder_path):
                parts = folder.split("_")
                if len(parts) >= 2:
                    code = parts[0]
                    receptor_name = parts[1].upper()  # e.g. convert "pparg" to "PPARG"
                    receptor_mapping[code] = receptor_name
                else:
                    print(f"Warning: unexpected receptor folder name format: {folder}")
    else:
        print(f"Warning: receptor directory {receptor_dir} not found.")
    
    # Rename DataFrame columns using the receptor mapping.
    df_pivot.rename(columns=receptor_mapping, inplace=True)
    
else:
    print("No binding energy data was extracted.")

100%|██████████| 36132/36132 [00:22<00:00, 1626.47it/s]


In [3]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm
from collections import Counter

def get_receptor_mapping(receptor_dir="AutodockGPU/receptor"):
    """
    Scan the receptor directory for folders and build a mapping:
       receptor_code -> receptor_name (converted to uppercase)
    """
    receptor_mapping = {}
    if os.path.exists(receptor_dir):
        for folder in os.listdir(receptor_dir):
            folder_path = os.path.join(receptor_dir, folder)
            if os.path.isdir(folder_path):
                parts = folder.split("_")
                if len(parts) >= 2:
                    code = parts[0]
                    receptor_name = parts[1].upper()
                    receptor_mapping[code] = receptor_name
                else:
                    print(f"Warning: unexpected receptor folder name format: {folder}")
    else:
        print(f"Warning: receptor directory {receptor_dir} not found.")
    return receptor_mapping

def parse_xml_contact_info(xml_file, receptor_mapping):
    """
    Parse an AutoDock-GPU XML file to extract contact analysis information.
    Compute each run’s weight as -free_NRG_binding, normalize weights to sum=1,
    then compute for each contact key the sum of normalized weights of runs
    where that contact appears.
    """
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
    except ET.ParseError as e:
        print(f"Error parsing {xml_file}: {e}")
        return None, None, None

    # Filename → receptor_code, ligand_code
    base_name = os.path.splitext(os.path.basename(xml_file))[0]
    parts = base_name.split("_")
    if len(parts) < 2:
        print(f"Warning: Unexpected filename format in {xml_file}")
        return None, None, None
    receptor_code = parts[0]
    ligand_code = parts[1]
    converted_receptor_name = receptor_mapping.get(receptor_code, receptor_code)

    runs_data = []
    for run in root.findall("runs/run"):
        free_elem = run.find("free_NRG_binding")
        if free_elem is None or free_elem.text is None:
            continue
        try:
            free_nrg = float(free_elem.text.strip())
        except ValueError:
            continue

        contact_analysis = run.find("contact_analysis")
        if contact_analysis is None:
            continue
        res_elem = contact_analysis.find("contact_analysis_residue")
        rid_elem = contact_analysis.find("contact_analysis_resid")
        if res_elem is None or rid_elem is None or res_elem.text is None or rid_elem.text is None:
            continue

        residues = [x.strip().strip('"') for x in res_elem.text.split(",") if x.strip()]
        resids   = [x.strip().strip('"') for x in rid_elem.text.split(",") if x.strip()]
        if len(residues) != len(resids):
            print(f"Warning: Mismatched lengths in {xml_file} run id {run.get('id')}")
            continue

        keys = { f"{converted_receptor_name}_{rid}_{res}"
                 for res, rid in zip(residues, resids) }
        runs_data.append({"contact_keys": keys, "free_nrg": free_nrg})

    if not runs_data:
        return receptor_code, ligand_code, {}

    # Compute raw weights = -free_nrg (so more negative free_nrg → larger positive weight)
    for d in runs_data:
        d["weight"] = -d["free_nrg"]

    # Normalize so sum(weights) = 1
    total = sum(d["weight"] for d in runs_data)
    if total <= 0:
        # fallback to equal weights if something went wrong
        n = len(runs_data)
        for d in runs_data:
            d["weight_norm"] = 1.0 / n
    else:
        for d in runs_data:
            d["weight_norm"] = d["weight"] / total

    # Build the set of all contact keys seen across runs
    all_keys = set().union(*(d["contact_keys"] for d in runs_data))

    # For each key, the weighted average indicator is just the sum of normalized weights
    # of runs where that key appears.
    contact_avg = {}
    for key in all_keys:
        contact_avg[key] = sum(d["weight_norm"] for d in runs_data if key in d["contact_keys"])

    return receptor_code, ligand_code, contact_avg

# Build receptor mapping
receptor_mapping = get_receptor_mapping()

# Gather XML files
input_dir = "AutodockGPU/adgpu_output"
xml_files = glob.glob(os.path.join(input_dir, "*.xml"))
if not xml_files:
    print(f"No XML files found in {input_dir}")

# Parse and accumulate per‐ligand
ligand_contact_results = {}
for xml_file in tqdm(xml_files, desc="Processing XML files"):
    receptor, ligand, contact_avg = parse_xml_contact_info(xml_file, receptor_mapping)
    if receptor is None or ligand is None or contact_avg is None:
        raise ValueError(f"Failed to parse contact info from {xml_file}")
    ligand_contact_results.setdefault(ligand, []).append(contact_avg)

# Combine across files: simple average of each contact key across all XMLs for that ligand
final_results = {}
for ligand, dicts in ligand_contact_results.items():
    concat_dict = {}
    for d in dicts:
        for key, val in d.items():
            assert key not in concat_dict, f"Duplicate key {key} found in ligand {ligand}"
            concat_dict[key] = val
    final_results[ligand] = concat_dict

# Make DataFrame: rows=ligand, cols=contact keys
all_keys = sorted({k for d in final_results.values() for k in d})
df_contact = pd.DataFrame(index=sorted(final_results), columns=all_keys, dtype=float)
for ligand, avg_dict in final_results.items():
    for key in all_keys:
        df_contact.at[ligand, key] = avg_dict.get(key, 0.0)

# Count columns per receptor
receptor_counts = Counter(col.split("_")[0] for col in df_contact.columns)
print("\nNumber of columns for each receptor:")
for receptor, cnt in sorted(receptor_counts.items()):
    print(f"{receptor}: {cnt}")


Processing XML files: 100%|██████████| 36132/36132 [00:39<00:00, 903.92it/s]



Number of columns for each receptor:
FXR: 56
LXRA: 50
LXRB: 45
PPARA: 57
PPARD: 58
PPARG: 55


In [4]:
df_pivot_z = df_pivot.copy()
for col in df_pivot_z.columns:
    mean = df_pivot_z[col].mean()
    std = df_pivot_z[col].std()
    df_pivot_z[col] = (df_pivot_z[col] - mean) / std
    
if not df_pivot_z.index.equals(df_contact.index):
    raise ValueError("Index mismatch between df_pivot_z and df_contact.")

df_concat = pd.concat([df_pivot_z, df_contact], axis = 1)
df_concat = df_concat.sort_index(axis=0)

df_concat

,PPARD,LXRA,LXRB,FXR,PPARA,PPARG,FXR_264_TYR,FXR_265_ASN,FXR_266_LYS,FXR_267_GLN,...,PPARG_367_LYS,PPARG_370_PHE,PPARG_371_ALA,PPARG_442_LEU,PPARG_445_ILE,PPARG_449_HIS,PPARG_453_LEU,PPARG_465_LEU,PPARG_469_LEU,PPARG_473_TYR
lig0001,1.567212,1.708113,1.436288,1.239629,1.467207,1.482036,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,1.000000,1.0
lig0003,1.078180,1.216345,0.991128,0.713336,1.167743,0.994735,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,1.000000,1.0
lig0004,1.315331,1.662367,1.551168,1.268706,1.585744,1.599773,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.049078,0.950922,0.950922,0.098698,1.0
lig0006,1.873593,2.085517,1.967608,1.745569,2.059896,1.949713,0.0,0.0,0.0,0.0,...,0.603522,0.603522,0.603522,0.0,0.603522,0.000000,0.000000,0.000000,0.000000,0.0
lig0007,1.272615,1.339287,1.078594,0.778759,1.359587,1.048698,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.048144,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lig7531,0.201753,0.324301,0.189579,0.486536,0.422202,0.191835,0.0,0.0,0.0,0.0,...,0.284516,0.000000,0.000000,0.0,0.000000,1.000000,0.101518,0.050406,0.329537,1.0
lig7532,0.888165,1.064812,0.891914,0.871806,1.018011,0.896621,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.899361,0.899361,0.899361,1.0
lig7533,-0.332942,-0.333296,-0.282996,-0.441019,-0.713266,-0.558738,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,1.000000,1.000000,1.0
lig7534,-0.321158,-0.487688,-1.068880,-0.752143,-0.335816,-0.499870,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [5]:
# save `df_concat` using joblib

import joblib

# make joblibdata directory if it doesn't exist
os.makedirs("joblibdata", exist_ok=True)

# Save the DataFrame to a file using joblib
file_path = os.path.join("joblibdata", "docking_result_dataframe.joblib")
joblib.dump(df_concat, file_path)

df_concat.to_excel("docking_result.xlsx", index=True)